In [1]:
import rclpy as rp
import jupyros.ros2 as jr2
import turtle_sim as turtle
from turtlesim.srv import Spawn
from turtlesim.msg import Pose
from time import time
import os
from std_msgs.msg import String
from geometry_msgs.msg import Twist
from sidecar import Sidecar
from time import time, sleep
import math



ModuleNotFoundError: No module named 'turtle_sim'

In [ ]:
# Initialize ROS communications for a given context
if(rp.ok() == False):
    rp.init()
else:
    print("rclpy already initiated")

In [ ]:
superturtle = rp.create_node("superturtle")
keyInput = rp.create_node("keyInput")
com = rp.create_node("com")
simple = rp.create_node("simple")
translate = rp.create_node("translate")

In [ ]:
key_send = jr2.KeyInput(keyInput, String, '/keyboard_stream')
key_send.display_inputs()

In [ ]:
turtlesim = turtle.TurtleSim(background_color="#0000FF")
display(turtlesim.canvas)

**TIP:** When using JupyterLab, you can right-click on the canvas and select *Create New View from Output*

In [ ]:
poses = {}

for name in turtlesim.turtles.keys():
    poses[name] = turtlesim.turtles[name].pose
    
print(poses["turtle1"])

In [ ]:
turtlesim.turtles[name].pose

In [ ]:
topic_name = '/keyboard_stream'
def move_turtles(msg):
    scale = 0.1
    name = "turtle1"

    poses[name] = {"x": turtlesim.turtles[name].pose["x"] + msg.linear.x*math.cos(turtlesim.turtles[name].pose["theta"])/scale,
                   "y": turtlesim.turtles[name].pose["y"] - msg.linear.x*math.sin(turtlesim.turtles[name].pose["theta"] )/scale,
                   "theta": turtlesim.turtles[name].pose["theta"] + msg.angular.z/ 180 * math.pi}
    
  
    turtlesim.move_turtles(new_poses=poses)




def cb(msg):
    # Receive string msg and translate
    
    key = msg.data
    
    #### General Keybindings  "key: (x, y, z, a (alpha), b (beta), g (gamma)"
    
    moveBindings = {
    'ArrowLeft': (0,0,0,0, 0, 1),
    'ArrowRight':(0,0,0,0, 0, -1),
    'ArrowUp': (1,0,0,0, 0, 0, 0),
    'ArrowDown': (-1,0,0,0, 0, 0)
    }

    """
    'i': (1, 0, 0, 0, 0, 0),
    'o': (1, 0, 0, -1, 0, 0)),
    """
    
    ## General Keybinding Decoder
    if key in moveBindings.keys():
        x = float(moveBindings[key][0])
        y = float(moveBindings[key][1])
        z = float(moveBindings[key][2])
        a = float(moveBindings[key][3])
        b = float(moveBindings[key][4])
        g = float(moveBindings[key][5])
        
    else:
        x = 0.0
        y = 0.0
        z = 0.0
        a = 0.0
        b = 0.0
        g = 0.0
        

   
    twist = Twist()
    twist.linear.x = x
    twist.angular.z = g 

    move_turtles(twist)


turtle_control = jr2.Subscriber(keyInput, String, topic_name, cb)
turtle_control.display()